In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility


In [2]:
sample = " if you want you"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex

print("idxToChar: ", idx2char)
print("charToIdx: ", char2idx)

idxToChar:  ['a', 'o', 'f', ' ', 'w', 'i', 'n', 't', 'u', 'y']
charToIdx:  {'a': 0, 'o': 1, 'f': 2, ' ': 3, 'w': 4, 'i': 5, 'n': 6, 't': 7, 'u': 8, 'y': 9}


In [3]:
# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

In [7]:
sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello
print("X_data: ", x_data)
print("Y_data: ", y_data)
print("X_shape: ", np.shape(x_data))
print("Y_shape: ", np.shape(y_data))

X_data:  [[3, 5, 2, 3, 9, 1, 8, 3, 4, 0, 6, 7, 3, 9, 1]]
Y_data:  [[5, 2, 3, 9, 1, 8, 3, 4, 0, 6, 7, 3, 9, 1, 8]]
X_shape:  (1, 15)
Y_shape:  (1, 15)


In [8]:
X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

print("X: ", X)
print("Y: ", Y)

X:  Tensor("Placeholder_2:0", shape=(?, 15), dtype=int32)
Y:  Tensor("Placeholder_3:0", shape=(?, 15), dtype=int32)


In [9]:
x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

In [10]:
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [11]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss:", l, "Prediction:", ''.join(result_str))

0 loss: 2.30917 Prediction:                
1 loss: 2.15889 Prediction:     uu u    uuu
2 loss: 2.04102 Prediction: y         y yo 
3 loss: 1.87853 Prediction: y    u    y you
4 loss: 1.66937 Prediction: y   uu   tt you
5 loss: 1.42024 Prediction: y  yuu   nt you
6 loss: 1.17605 Prediction: yy you uant you
7 loss: 0.951468 Prediction: yf you aatt you
8 loss: 0.736741 Prediction: yf you aant you
9 loss: 0.569355 Prediction: yf you want you
10 loss: 0.434493 Prediction: ff you want you
11 loss: 0.334538 Prediction: ff you want you
12 loss: 0.252639 Prediction: ff you want you
13 loss: 0.1903 Prediction: if you want you
14 loss: 0.146655 Prediction: if you want you
15 loss: 0.113104 Prediction: if you want you
16 loss: 0.0831911 Prediction: if you want you
17 loss: 0.0611577 Prediction: if you want you
18 loss: 0.0467486 Prediction: if you want you
19 loss: 0.0359255 Prediction: if you want you
20 loss: 0.0267404 Prediction: if you want you
21 loss: 0.0193222 Prediction: if you want you
2